# **Chatbot**

Este proyecto es para crear un chatbot conversacional usando Sequence para secuenciar modelos LSTM. El aprendizaje de secuencia a secuencia consiste en entrenar modelos para convertir de un dominio a secuencias en otro dominio.


# Step 1: Import all the packages 

In [ ]:
import numpy as np 
import tensorflow as tf
import pickle
from tensorflow.keras import layers, activations, models, preprocessing

# Step 2: Download all the data from kaggle

In [ ]:
!pip install kaggle 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

Saving chatterbotenglish.zip to chatterbotenglish.zip


{'chatterbotenglish.zip': b'Rar!\x1a\x07\x01\x00\xf3\xe1\x82\xeb\x0b\x01\x05\x07\x00\x06\x01\x01\x80\x80\x80\x00\xb8\xfa\x8d\x823\x02\x03\x0b\xe2\x99\x00\x04\xd0\xc7\x00 \xbe\xdf\xea\xd3\x80\x03\x00\x15ChatBotSpanish/ai.yml\n\x03\x02$\xcd\x93\x16\xb3\n\xd9\x01\xc9A\xde\x0cG\x05T334\xf2gE\xde\xfc(I\xa5\xe9Xk6\xe6itq\xb5\x95\x15SN&\xd2t\xe6l\xb2\x05\x92B\x80\x8b2\xc0\x05\xb2\xfc\x1f<!lz\x16\x8c\xa7\xad\xd6UJ\xa6<\xcaBW9\xdb\xf8\x00\x0b)\x12\x9dO\x0c\xcc\x91 \xae\xfb\xf3\xbe\xfco\xb7<k\xbf\xc2~\xe7/\x97\xa6\xef\xc5eUk\xcc\xc6\xce\xba\xbf\xcf\x93\xfbI\x8f\x98\xfe\x14\xda\xe7\xcb<\xb3\xaa\x89\'\xa6\xb5\xd1D\xf9\x97L\xab\xf2J\xcau-\xb5\x01\t\xc7\xc3\x83G\xff\xfa\xfe\xfe\xbb-\xf9H\x07$\xa1Rsc\xf3y$\x90\xea\xfaO\xda\x0c8\\\x11\x81V\xf1A\xe1-V\x81\xf8\x99i\xf4\x0c$\x10\x1f[oh\xfb\xd6\x9a\x00\x19\xe2\x83z`$\xa9\x87\xc1\xd2&\xfa\xdbd\xed\x93E\xbd\x9f\xb5\x9d\xc5l\x0f\xd0\x1e\xd2B\x82\x1dN.\n\x15\x8d5\x02\x1a\x08\xc0\xeb\xa5\x90;D\xcb\xddj\x1f\xe3%\xb3J\x1c\xaf\x9a\xb3\xa9\xb3{\xc7\xfa\xb2\x8a\xd4

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!ls ~/.kaggle

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d kausr25/chatterbotenglish

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip /content/chatterbotenglish.zip

Archive:  /content/chatterbotenglish.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/chatterbotenglish.zip or
        /content/chatterbotenglish.zip.zip, and cannot find /content/chatterbotenglish.zip.ZIP, period.


In [ ]:
!wget https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true -O chatbot_nlp.zip
!unzip chatbot_nlp.zip

--2022-12-14 16:10:25--  https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/shubham0204/Dataset_Archives/raw/master/chatbot_nlp.zip [following]
--2022-12-14 16:10:25--  https://github.com/shubham0204/Dataset_Archives/raw/master/chatbot_nlp.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/chatbot_nlp.zip [following]
--2022-12-14 16:10:26--  https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/chatbot_nlp.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

# Step 3: Preprocessing the data

### a)Leer los datos de los archivos.
Analizamos cada uno de los archivos .yaml.

1. Leer los datos de los archivos.
Analizamos cada uno de los archivos .yaml.
2. Elimine los tipos de datos no deseados que se producen al analizar los datos.
3. Agregue <START> y <END> a todas las respuestas.
4. Cree un Tokenizer y cargue todo el vocabulario (preguntas + respuestas) en él.

In [ ]:
from tensorflow.keras import preprocessing, utils
import os
import yaml

El conjunto de datos contiene archivos .yml que tienen pares de preguntas diferentes y sus respuestas sobre temas variados como historia, perfil de bot, ciencia, etc.
Podemos leerlos fácilmente de la siguiente manera:

In [24]:
dir_path = '/content/chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)

In [25]:
questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 1973


### b) Preparación de datos para el modelo Seq2Seq

Este modelo requiere 3 matrices encoder_input_data, decoder_input_data y decoder_output_data.

Para encoder_input_data:
Tokenize las preguntas y rellénelas a su longitud máxima.

Para decoder_input_data:
Tokenize las respuestas y rellénelas a su longitud máxima.

Para decoder_output_data:
Tokenize the Answers y elimine el primer elemento de todas las tokenized_answers. Este es el elemento <START> que se agregó anteriormente.

In [26]:
from gensim.models import Word2Vec
import re

In [27]:
vocab = []
for word in tokenizer.word_index:
  vocab.append(word)

def tokenize(sentences):
  tokens_list = []
  vocabulary = []
  for sentence in sentences:
    sentence = sentence.lower()
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    tokens = sentence.split()
    vocabulary += tokens
    tokens_list.append(tokens)
  return tokens_list, vocabulary

In [28]:
#encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)

(543, 25) 25


In [29]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

(543, 184) 184


In [30]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(543, 184, 1973)


# Step 4: Defining Encoder Decoder Model





In [31]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 25)]         0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 184)]        0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 25, 200)      394600      ['input_5[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, 184, 200)     394600      ['input_6[0][0]']                
                                                                                            

# Step 5: Training the Model

Entrenamos el modelo para varias épocas con el optimizador RMSprop y la función de pérdida de categorical_crossentropy.

In [32]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 ) 
model.save( 'model.h5' )

Epoch 1/150
11/11 [==============================] - 26s 1s/step - loss: 0.5168
Epoch 2/150
11/11 [==============================] - 16s 1s/step - loss: 0.4419
Epoch 3/150
11/11 [==============================] - 16s 1s/step - loss: 0.4321
Epoch 4/150
11/11 [==============================] - 16s 1s/step - loss: 0.4250
Epoch 5/150
11/11 [==============================] - 16s 1s/step - loss: 0.4176
Epoch 6/150
11/11 [==============================] - 16s 1s/step - loss: 0.4105
Epoch 7/150
11/11 [==============================] - 16s 2s/step - loss: 0.4046
Epoch 8/150
11/11 [==============================] - 16s 1s/step - loss: 0.3986
Epoch 9/150
11/11 [==============================] - 16s 1s/step - loss: 0.3935
Epoch 10/150
11/11 [==============================] - 16s 1s/step - loss: 0.3884
Epoch 11/150
11/11 [==============================] - 16s 1s/step - loss: 0.3837
Epoch 12/150
11/11 [==============================] - 17s 2s/step - loss: 0.3790
Epoch 13/150
11/11 [=================

# Step 6: Defining Inference Models

Modelo de inferencia de codificador: toma preguntas como entrada y salida de estados LSTM (h y c)

Modelo de inferencia del decodificador: toma 2 entradas, una son los estados LSTM, la segunda son las secuencias de entrada de respuesta. abrirá las respuestas a las preguntas que alimentaron el modelo del codificador y sus valores de estado.

In [33]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

# Step 7: Talking with the Chatbot

defina un método str_to_tokens que convierta preguntas str en tokens Integer con relleno.

1. Primero, tomamos una pregunta como entrada y predecimos los valores de estado usando enc_model.
2. Establecemos los valores de estado en el LSTM del decodificador.
3. Luego, generamos una secuencia que contiene el elemento <start>.
4. Ingresamos esta secuencia en dec_model.
5. Reemplazamos el elemento <start> con el elemento que fue predicho por dec_model y actualizamos los valores de estado.
6. Llevamos a cabo los pasos anteriores de forma iterativa hasta que lleguemos a la etiqueta <end> o la longitud máxima de respuesta.

In [34]:
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()
  
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Ingrese pregunta : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Ingrese pregunta : Hola
1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 0s 23ms/step
 hola end
Ingrese pregunta : como estas
1/1 [==============================] - 0s 26ms/step
 lo estoy haciendo bien como estas end
Ingrese pregunta : que es illuminatti
1/1 [==============================] - 0s 23ms/step
 una supuesta conspiracion mundial end


# Conversion to TFLite 

Podemos convertir nuestro modelo seq2seq en un modelo TensorFlow Lite para que podamos usarlo en dispositivos perimetrales

In [ ]:
!pip install tf-nightly

In [41]:
converter = tf.lite.TFLiteConverter.from_keras_model( enc_model )
buffer = converter.convert()
open( 'enc_model.tflite' , 'wb' ).write( buffer )

converter = tf.lite.TFLiteConverter.from_keras_model( dec_model )
open( 'dec_model.tflite' , 'wb' ).write( buffer )

KeyboardInterrupt: ignored